In [1]:
from telethon.sync import TelegramClient
from telethon.tl.functions.messages import GetHistoryRequest
import nest_asyncio
nest_asyncio.apply()
import json
import os
import pandas as pd

api_id = 22723598      
api_hash = 'c2647da4d95bd4983ade135744857eda'
phone_number = '+251912356409'     
session_name = 'ethio_telegram_scraper'
channel_username = [
    'ZemenExpress',
    'nevacomputer',
    'meneshayeofficial',
    'ethio_brand_collection',
    'Leyueqa'
]
async def scrape_channels_to_csv():
    all_messages = []

    async with TelegramClient(session_name, api_id, api_hash) as client:
        await client.start(phone=phone_number)

        for username in channel_username:
            try:
                entity = await client.get_entity(username)
                history = await client(GetHistoryRequest(
                    peer=entity,
                    limit=2000,
                    offset_date=None,
                    offset_id=0,
                    max_id=0,
                    min_id=0,
                    add_offset=0,
                    hash=0
                ))

                for message in history.messages:
                    if message.message:
                        all_messages.append({
                            'channel': username,
                            'message_id': message.id,
                            'text': message.message,
                            'date': str(message.date),
                            'sender_id': message.from_id.user_id if message.from_id else None
                        })

                print(f"✅ Scraped from {username}")
            except Exception as e:
                print(f"⚠️ Failed to scrape {username}: {e}")
   
    df = pd.DataFrame(all_messages)
    df.to_csv('all_telegram_messages.csv', index=False, encoding='utf-8-sig')
    print(" All messages saved to all_telegram_messages.csv")

In [2]:
await scrape_channels_to_csv()

✅ Scraped from ZemenExpress
✅ Scraped from nevacomputer
✅ Scraped from meneshayeofficial
✅ Scraped from ethio_brand_collection
✅ Scraped from Leyueqa
 All messages saved to all_telegram_messages.csv


In [3]:
df = pd.read_csv('C:\\Users\\Hiwi\\Documents\\week4\\all_telegram_messages.csv')

In [5]:
import re
from etnltk import Amharic
import unicodedata
from etnltk.tokenize.am import sent_tokenize
from etnltk.tokenize.am import word_tokenize
# === Abbreviation and Location Mapping ===
LOCATION_MAPPING = {
    'addis ababa': ('አዲስ', 'አበባ'), 'አዲስ አበባ': ('አዲስ', 'አበባ'),
    'bahir dar': ('ባህር', 'ዳር'), 'ባህር ዳር': ('ባህር', 'ዳር'),
    'dire dawa': ('ድሬ', 'ዳዋ'), 'ድሬ ዳዋ': ('ድሬ', 'ዳዋ'),
    'አዳማ': ('አዳማ',), 'ጅማ': ('ጅማ',), 'መቀሌ': ('መቀሌ',), 'ሀዋሳ': ('ሀዋሳ',),
    'ቦሌ': ('ቦሌ',), 'ሰበር ምድር': ('ሰበር', 'ምድር'), 'አቃቂ': ('አቃቂ',),
    'ጎንደር': ('ጎንደር',), 'ደሴ': ('ደሴ',), 'አርባ ምንጭ': ('አርባ', 'ምንጭ'),
    'ነቀምት': ('ነቀምት',), 'ጂንማ': ('ጂንማ',)
}

ABBREVIATION_MAP = {
    'ቢሮ': 'ቢሮዎች', 'ም/ቤት': 'ምክር ቤት', 'ወ/ሮ': 'ወይዘሮ', 'ወ/ሪት': 'ወይዘሪት',
    'ኪ.ሜ': 'ኪሎ ሜትር', 'ቁ.ቁ': 'ቁጥር ቁጥር', 'ቁ': 'ቁጥር', 'ሜ': 'ሜትር',
    'አ.አ': 'አዲስ አበባ', 'ሺ': 'ሺህ', 'ኪ.ግ': 'ኪሎ ግራም'
}
#  cleaning
def clean_amharic_text(text):
    if not isinstance(text, str):
        return ""  # skip non-string entries
    text = re.sub(r'[^\w\s\u1200-\u137F.,!?፡።፣]', '', text)
    replacements = {
        '::': '።',
        ':': '፡',
        ',': '፣'
    }
    for k, v in replacements.items():
        text = text.replace(k, v)
    text = re.sub(r'\s+', ' ', text).strip()
    text = unicodedata.normalize('NFC', text)
    return text
# Remove emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub('', text)
    
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
# 2. Tokenize Amharic sentences
def tokenize_amharic(text):
    return sent_tokenize(text)
# Load the dataset
df = pd.read_csv('C:\\Users\\Hiwi\\Documents\\week4\\all_telegram_messages.csv')  

# Clean the 'message' column
df['cleaned_message'] = df['text'].apply(clean_amharic_text)

# Optional: Save the cleaned data to a new file
df.to_csv('data/cleaned_telegram_ecommerce.csv', index=False)

# 5. Apply tokenization
df['tokenized_sentences'] = df['cleaned_message'].apply(tokenize_amharic)
def expand_abbreviations(tokens):
    return [ABBREVIATION_MAP.get(token, token) for token in tokens]

# === Step 5: Normalize Location Names ===
def normalize_locations(tokens):
    normalized = []
    i = 0
    while i < len(tokens):
        found = False
        for length in range(3, 0, -1):
            if i + length <= len(tokens):
                phrase = ' '.join(tokens[i:i+length]).lower()
                if phrase in LOCATION_MAPPING:
                    normalized.extend(LOCATION_MAPPING[phrase])
                    i += length
                    found = True
                    break
        if not found:
            normalized.append(tokens[i])
            i += 1
    return normalized

# === Process Tokens ===
df['word_tokens'] = df['cleaned_message'].apply(word_tokenize)
df['expanded_tokens'] = df['word_tokens'].apply(expand_abbreviations)
df['location_normalized_tokens'] = df['expanded_tokens'].apply(normalize_locations)

# === Save Processed Data ===
df.to_csv('data/processed_telegram_ecommerce.csv', index=False)


# 7. Preview
print(df[['text', 'cleaned_message', 'tokenized_sentences', 'word_tokens', 'location_normalized_tokens']].head())

                                                text  \
0  💥💥...................................💥💥\n\n📌Sa...   
1  💥💥...................................💥💥\n\n3pc...   
2  💥💥...................................💥💥\n\n3pc...   
3  💥💥...................................💥💥\n\n📌1 ...   
4  💥💥...................................💥💥\n\n📌1 ...   

                                     cleaned_message  \
0  ................................... Saachi Ele...   
1  ................................... 3pcs Bottl...   
2  ................................... 3pcs Bottl...   
3  ................................... 1 pairs Sn...   
4  ................................... 1 pairs Sn...   

                                 tokenized_sentences  \
0  [ዋጋ ብር ውስን ፍሬ ነው ያለው አድራሻ መገናኛመሰረትደፋርሞልሁለተኛፎቅ ...   
1  [በማንኛውም ጠርሙስ ጫፍ የሚገጠም ለዘይት እና መሰል ነገሮች መቀነሻ የሚ...   
2  [በማንኛውም ጠርሙስ ጫፍ የሚገጠም ለዘይት እና መሰል ነገሮች መቀነሻ የሚ...   
3  [ዋጋ ብር ውስን ፍሬ ነው ያለው አድራሻ መገናኛመሰረትደፋርሞልሁለተኛፎቅ ...   
4  [ዋጋ ብር ውስን ፍሬ ነው ያለው አድራሻ መገናኛመሰረትደፋርሞልሁለተኛ

In [ ]:
import pandas as pd
import re
import unicodedata

# Load your scraped dataset
df = pd.read_csv('C:\\Users\\Hiwi\\Documents\\week4\\all_telegram_messages.csv')

# Define your labeling function
def label_message(message, tokens):
    print("\n" + "="*80)
    print(f"Original Message: {message}")
    print("-"*80)
    print(f"Tokens: {tokens}")
    
    labels = []
    for i, token in enumerate(tokens):
        valid_labels = ['B-PROD', 'I-PROD', 'B-LOC', 'I-LOC', 'B-PRICE', 'I-PRICE', 'O']
        
        # Show context tokens
        context_start = max(0, i-2)
        context_end = min(len(tokens), i+3)
        context = tokens[context_start:context_end]
        
        print("\n" + "-"*40)
        print(f"Context: {' '.join(context)}")
        print(f"Current Token ({i+1}/{len(tokens)}): '{token}'")
        
        while True:
            label = input("Label: ").strip().upper()
            if label in valid_labels:
                labels.append((token, label))
                break
            print(f"Invalid label! Please use one of: {', '.join(valid_labels)}")
    
    return labels

# Define function to save to CoNLL format
def save_to_conll(labeled_data, output_path):
    with open(output_path, 'w', encoding='utf-8') as f:
        for original_msg, annotations in labeled_data:
            for token, label in annotations:
                f.write(f"{token}\t{label}\n")
            f.write("\n")  # Blank line between messages
    print(f"Saved {len(labeled_data)} messages to {output_path}")

# Precompute tokens for all messages
df['tokens'] = df['text'].apply(clean_amharic_text)

# Select messages to label (first 10)
messages_to_label = df.head(10)

labeled_data = []
for idx, row in messages_to_label.iterrows():
    message = row['text']
    tokens = row['tokens']
    
    print(f"\nProcessing message {idx+1}/{len(messages_to_label)}")
    annotations = label_message(message, tokens)
    labeled_data.append((message, annotations))
    
    print(f"Message labeled! ({len(labeled_data)}/{len(messages_to_label)} done)")

print(f"\nSuccessfully labeled {len(labeled_data)} messages!")

# Now save to CoNLL format
save_to_conll(labeled_data, "my_labeled_dataset.conll")

# Verify the output
print("\nPreview of labeled data:")
with open("my_labeled_dataset.conll", 'r', encoding='utf-8') as f:
    print(f.read(500))  # Print first 500 characters


Processing message 1/10

Original Message: 💥💥...................................💥💥

📌Saachi Electric Kettle

👍Borosilicate Glass Body
👍Overheat protection
👍Automatic switch off
👍2200w

ዋጋ፦  💲🏷 2700  ብር ✅

♦️ውስን ፍሬ ነው ያለው🔥🔥🔥

🏢 አድራሻ👉

📍♦️#መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ. S05/S06


     💧💧💧💧


    📲 0902660722
    📲 0928460606 

🔖
💬በTelegram ለማዘዝ ⤵️ ይጠቀሙ🔽

@zemencallcenter 

@zemenexpressadmin

ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን⤵️
https://telegram.me/zemenexpress

💥💥...................................💥💥
--------------------------------------------------------------------------------
Tokens: ................................... Saachi Electric Kettle Borosilicate Glass Body Overheat protection Automatic switch off 2200w ዋጋ፦ 2700 ብር ውስን ፍሬ ነው ያለው አድራሻ መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ ቢሮ ቁ. S05S06 0902660722 0928460606 በTelegram ለማዘዝ ይጠቀሙ zemencallcenter zemenexpressadmin ለተጨማሪ ማብራሪያ የቴሌግራም ገፃችን httpstelegram.mezemenexpress ...................................

----------------------------------------
Context: .